In [87]:
%load_ext iminizinc

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns

# import mplfinance as mpf
%matplotlib inline

mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})
plt.rcParams['lines.linewidth'] = 0.5 



The iminizinc extension is already loaded. To reload it, use:
  %reload_ext iminizinc


In [88]:

def plot_candles(wdf, ax=None, kwargs={}):
    if ax is None:
        fig, ax = plt.subplots(**kwargs)

    up, down = wdf[wdf.close >= wdf.open], wdf[wdf.close < wdf.open]
    col1,col2 = 'green','red'
    width, width2 = .1, .02
    # Plotting up prices of the stock
    ax.bar(up.index, up.close-up.open, width, bottom=up.open, color=col1)
    ax.bar(up.index, up.high-up.close, width2, bottom=up.close, color=col1)
    ax.bar(up.index, up.low-up.open, width2, bottom=up.open, color=col1)
    # Plotting down prices of the stock
    ax.bar(down.index, down.close-down.open, width, bottom=down.open, color=col2)
    ax.bar(down.index, down.high-down.open, width2, bottom=down.open, color=col2)
    ax.bar(down.index, down.low-down.close, width2, bottom=down.close, color=col2)    
    plt.xticks(rotation=30, ha='right')
    
def load_candles(pair, timeframe):
    odf = pd.read_json(f'../../freq-user-data/data/binance/{pair}-{timeframe}.json').dropna()
    odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

    odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
    odf['idate'] = odf.date.dt.strftime('%Y%m%d')
    odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
    odf = odf.sort_index()
    return odf



def make_indicators(wdf, hold, lag, ):
    wdf['rfd'] = wdf.low.rolling(hold).min().shift(-hold)
    wdf['rfu'] = wdf.high.rolling(hold).max().shift(-hold)
    wdf['lr_fu'] = np.log(wdf.rfu / wdf.close)
    wdf['lr_fd'] = np.log(wdf.rfd / wdf.close)
    
    wdf=wdf.join(wdf[['lr_fu', 'lr_fd']].apply(lambda x: x.multiply(100).round(2)).add_prefix('p'))
    return wdf

def call_mzn_model(model_name,model_params):
    with open(f'pars-{model_name}.dzn.json', 'w+') as f: f.write(json.dumps(model_params, indent=2))

    # print('Model params:', model_params)
    mzn_model = Model(f'{model_name}.mzn')
    gecode = Solver.lookup("gecode")
    instance = Instance(gecode, mzn_model)

    for k,v in model_params.items(): instance[k] = v
    result = instance.solve()

    return result


## 7 Day returns after a long upper wick

In [91]:
wdf = odf.copy()

In [98]:
threshold = 0.01
wdf['ret'] = wdf.close.divide(wdf.close.shift()).apply(np.log)
wdf['btop'] = wdf[['open', 'close']].max(axis=1)
wdf['bbot'] = wdf[['open', 'close']].min(axis=1)


wdf

,timestamp,open,high,low,close,volume,date,idate,ret,btop,bbot
date,,,,,,,,,,,
2020-01-01,1577836800000,7195.24,7255.00,7175.15,7200.85,16792.388165,2020-01-01,20200101,NaN,7200.85,7195.24
2020-01-02,1577923200000,7200.77,7212.50,6924.74,6965.71,31951.483932,2020-01-02,20200102,-0.033200,7200.77,6965.71
2020-01-03,1578009600000,6965.49,7405.00,6871.04,7344.96,68428.500451,2020-01-03,20200103,0.053015,7344.96,6965.49
2020-01-04,1578096000000,7345.00,7404.00,7272.21,7354.11,29987.974977,2020-01-04,20200104,0.001245,7354.11,7345.00
2020-01-05,1578182400000,7354.19,7495.00,7318.00,7358.75,38331.085604,2020-01-05,20200105,0.000631,7358.75,7354.19
...,...,...,...,...,...,...,...,...,...,...,...
2023-05-28,1685232000000,26854.28,28261.32,26764.36,28065.00,43916.008550,2023-05-28,20230528,0.044098,28065.00,26854.28
2023-05-29,1685318400000,28065.01,28447.14,27524.60,27736.40,42385.419450,2023-05-29,20230529,-0.011778,28065.01,27736.40
2023-05-30,1685404800000,27736.39,28038.59,27554.00,27694.40,32686.753710,2023-05-30,20230530,-0.001515,27736.39,27694.40


In [97]:
wdf[['open', 'close']].min(axis=1)

date
2020-01-01     7195.24
2020-01-02     6965.71
2020-01-03     6965.49
2020-01-04     7345.00
2020-01-05     7354.19
                ...   
2023-05-28    26854.28
2023-05-29    27736.40
2023-05-30    27694.40
2023-05-31    27210.35
2023-06-01    26817.93
Length: 1248, dtype: float64

## Upper wicks, wick/body ratio, next *hold* periods return

In [89]:
odf = load_candles('BTC_USDT', '1d')
wdf = odf.copy()

In [75]:
pers = [1,3,5,5,7,10,14,30]
df = wdf.head(100)
idf = pd.DataFrame({'lret': df.close.divide(df.close.shift()).apply(np.log)}
    )#.assign(ret=lambda df:df.lret.apply(np.expm1))
idf = idf.assign(mret=lambda df: df.lret.multiply(1000).round().astype('Int64'))
idf = idf.join(pd.DataFrame({
    f'min_c_{per}': df.close.rolling(per+1).min()
    for per in pers
})).join(pd.DataFrame({
    f'max_c_{per}': df.close.rolling(per+1).max()
    for per in pers
})).join(pd.DataFrame({
    f'max_c_{per}': df.close.rolling(per+1).max()
    for per in pers
}))
idf = idf.join(pd.DataFrame({
    f'lret_{per}': idf.lret.rolling(per).sum()
    for per in pers
}))
# idf = idf.join(pd.DataFrame({
#     f'mret_{per}': idf[f'ret_{per}'].multiply(1000).round().astype('Int64')
#     for per in pers
# }))
idf = idf.join(pd.DataFrame({
    f'mlret_{per}': idf[f'lret_{per}'].multiply(1000).round().astype('Int64')
    for per in pers
}))
idf = idf.join(pd.DataFrame({
    f'fu_{per}_h_max': df.high.rolling(per).max().shift(-per)
    for per in pers
})).join(pd.DataFrame({
    f'fu_{per}_l_min': df.low.rolling(per).min().shift(-per)
    for per in pers
}))
# idf = idf.join(pd.DataFrame({

#     f'fu_ret_{per}': idf[f'ret_{per}'].shift(-per)
#     for per in pers
# }))
# idf = idf.join(pd.DataFrame({

#     f'fu_ret_{per}': idf[f'ret_{per}'].shift(-per)
#     for per in pers
# }))

idf
# pd.DataFrame()

,lret,mret,min_c_1,min_c_3,min_c_5,min_c_7,min_c_10,min_c_14,min_c_30,max_c_1,...,fu_10_h_max,fu_14_h_max,fu_30_h_max,fu_1_l_min,fu_3_l_min,fu_5_l_min,fu_7_l_min,fu_10_l_min,fu_14_l_min,fu_30_l_min
date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8455.00,8916.48,9578.00,6924.74,6871.04,6871.04,6871.04,6871.04,6871.04,6871.04
2020-01-02,-0.033200,-33,6965.71,NaN,NaN,NaN,NaN,NaN,NaN,7200.85,...,8455.00,8916.48,9578.00,6871.04,6871.04,6871.04,6871.04,6871.04,6871.04,6871.04
2020-01-03,0.053015,53,6965.71,NaN,NaN,NaN,NaN,NaN,NaN,7344.96,...,8455.00,9041.65,9578.00,7272.21,7272.21,7272.21,7272.21,7272.21,7272.21,7272.21
2020-01-04,0.001245,1,7344.96,6965.71,NaN,NaN,NaN,NaN,NaN,7354.11,...,8880.00,9041.65,9618.79,7318.00,7318.00,7318.00,7318.00,7318.00,7318.00,7318.00
2020-01-05,0.000631,1,7354.11,6965.71,NaN,NaN,NaN,NaN,NaN,7358.75,...,8916.48,9198.98,9618.79,7346.76,7346.76,7346.76,7346.76,7346.76,7346.76,7346.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-05,-0.012357,-12,6772.78,6734.10,6410.44,5881.42,5881.42,5816.19,4800.0,6856.99,...,NaN,NaN,NaN,6765.00,6765.00,NaN,NaN,NaN,NaN,NaN
2020-04-06,0.079050,79,6772.78,6734.10,6642.92,6394.38,5881.42,5881.42,4800.0,7329.90,...,NaN,NaN,NaN,7077.00,7077.00,NaN,NaN,NaN,NaN,NaN
2020-04-07,-0.018253,-18,7197.32,6772.78,6734.10,6410.44,5881.42,5881.42,4800.0,7329.90,...,NaN,NaN,NaN,7150.00,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
idf['2020-03-01':].max_c_30.head(20)

date
2020-03-01    10344.36
2020-03-02    10344.36
2020-03-03    10344.36
2020-03-04    10344.36
2020-03-05    10344.36
2020-03-06    10344.36
2020-03-07    10344.36
2020-03-08    10344.36
2020-03-09    10344.36
2020-03-10    10344.36
2020-03-11    10344.36
2020-03-12    10344.36
2020-03-13    10344.36
2020-03-14    10344.36
2020-03-15    10344.36
2020-03-16    10164.71
2020-03-17    10164.71
2020-03-18    10164.71
2020-03-19    10164.71
2020-03-20     9936.40
Name: max_c_30, dtype: float64

In [ ]:

### Ranges
nest_asyncio.apply()

strategy = 'stats-ranges'
strategy_params_json = f'./par-{strategy}.json'

mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})

dlen = len(odf)


sl_n2= IntSlider(description="n2", min=0, max=ceil(log(dlen, 2.0)+1), step=1, value=9)
sl_w = IntSlider(description="w", min=0, max=ceil(dlen/pow(2,sl_n2.value-1)-1), step=1, value=1)
def update_sl_w_range(*args):
    sl_w.max = ceil(dlen/pow(2,sl_n2.value-1)-1)
sl_n2.observe(update_sl_w_range, 'value')

sl_hold = IntSlider(value=7,min=1,max=50,step=1,description='Hold',continuous_update=False)
sl_lag_n = IntSlider(value=5,min=1,max=50,step=1,description='lag_n',continuous_update=False)
# tx_max_rat = FloatText(value=0.05,min=0.01,max=0.1,step=0.001,description='max_rat',continuous_update=False)
sl_rat = FloatRangeSlider(value=(0.02,0.05),min=0.01,max=0.2,step=0.001,description='sl_rat',continuous_update=False)
chk_candles = Checkbox(value=True, description='Candles', disabled=False)
dd_dir = Dropdown(options=[('Up', 1), ('Down', -1)], value=1, description='Dir',)

strategy_params = {
    "hold": {'wdg': sl_hold},
    "candles": {"wdg": chk_candles},
    "mdir": {"wdg": dd_dir},
    "lag_n": {"wdg": sl_lag_n},
    # "max_rat": {"wdg": tx_max_rat},
    "rat": {"wdg": sl_rat}
}
all_params = {
    'w2log': {'wdg': sl_n2}, 'w': {'wdg': sl_w},
    **strategy_params
}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([
    widgets.HBox([sl_n2, sl_w]),
    widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])
])

if os.path.exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in all_params.items(): 
            if k in js: v['wdg'].value=js[k];

else: print(f'File not found: {strategy_params_json}')

wdf,glag_n, min_rat, max_rat = 0,0,0,0
mzn_result = None
sdf, spa_mins, spa_maxs, i_spa_begs, i_spa_sigs, ix_spa_begs, ix_spa_sigs = 0,0,0,0,0,0,0
i_hi_ends, i_lo_ends, ix_hi_ends, ix_lo_ends = 0,0,0,0
def printer(
        w2log, w, hold, candles, mdir, lag_n, rat
):
    
    global wdf,glag_n, min_rat, max_rat
    global mzn_result
    global sdf, spa_mins, spa_maxs, i_spa_begs, i_spa_sigs, ix_spa_begs, ix_spa_sigs
    global i_hi_ends, i_lo_ends, ix_hi_ends, ix_lo_ends

    glag_n = lag_n
    min_rat, max_rat = rat
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))
    
    
    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
    
    wdf = odf.iloc[wst:wed,:].copy()    
    wlen = len(wdf)

    print(f'N={len(wdf)}; Period: {wdf.index[-1] - wdf.index[0]}, Start: {wdf.index[0]}, End: {wdf.index[-1]}\n')
    # print(f'lag_n={lag_n}; min_rat={min_rat}; max_rat={max_rat}\n')

    sdf = pd.DataFrame({'close': wdf.close.astype(int)}
    ).assign(ba_max=lambda x: x['close'].rolling(lag_n).max().bfill().astype(int), ba_min = lambda x: x['close'].rolling(lag_n).min().bfill().astype(int)
    ).assign(ba_rat = lambda df:df['ba_max']/df['ba_min'] - 1
    ).assign(spa_in=lambda x:((x['ba_rat'] >= min_rat) & (x['ba_rat'] <= max_rat))
    ).assign(spa_sig=lambda x: x['spa_in'] & (~x['spa_in']).shift()
    ).assign(spa_beg=lambda x:x['spa_sig'].shift(-(lag_n-1))
    )
    sdf.to_csv('test.csv')

    i_spa_begs, i_spa_sigs = [sdf[c].fillna(False).values.nonzero()[0] for c in ['spa_beg','spa_sig']]
    ix_spa_begs, ix_spa_sigs = [wdf.index[c] for c in [i_spa_begs, i_spa_sigs]]
    spa_mins = [int(sdf.loc[ix_spa_begs[i]:ix_spa_sigs[i]].close.min()) for i in range(len(ix_spa_sigs))]
    spa_maxs = [int(sdf.loc[ix_spa_begs[i]:ix_spa_sigs[i]].close.max()) for i in range(len(ix_spa_sigs))]
    # if len(spa_mins) > 0: spa_mins[-1] = int(sdf.close.max())
    # if len(spa_maxs) > 0: spa_maxs[-1] = int(sdf.close.min())
    # print(f'i_spa_begs={i_spa_begs}, i_spa_sigs={i_spa_sigs}\n')
    # print(f'spa_mins={spa_mins}, spa_maxs={spa_maxs}\n')
    rats = np.array([])
    if len(ix_spa_sigs) > 0:
        rats = np.array(spa_maxs) / spa_mins - 1
        # , rats={rats}
        print(f'Spans={len(ix_spa_sigs)}, ratsrange={rats.min(),rats.max()}\n')
    else:
        print('No spans')

    ###/ MiniZinc to find span endings
    # min_close, max_close
    rsdf = sdf.loc[::-1]    
    fu_mins, fu_maxs = rsdf.close.cummin()[::-1], rsdf.close.cummax()[::-1]
    fu_mins[-1] = int(sdf.close.min())
    fu_maxs[-1] = int(sdf.close.max())
    mzn_result = call_mzn_model('span-endings',{
        'i_spa_sigs': (i_spa_sigs+1).tolist(),
        'closes' : sdf.close.values.tolist(),
        'ba_mins' : sdf.ba_min.bfill().values.tolist(),
        'ba_maxs' : sdf.ba_max.bfill().values.tolist(),
        'spa_mins': spa_mins, 
        'spa_maxs': spa_maxs,
        'cmin': int(sdf.close.min()), 'cmax': int(sdf.close.max()),
        'min_to_end': fu_mins.values.tolist(), 'max_to_end': fu_maxs.values.tolist()
    })
    print(f'mzn_result: {mzn_result}')
    ix_hi_ends, ix_lo_ends = [], []
    if mzn_result.status == mzStatus.SATISFIED:
        i_lo_ends = (np.array(mzn_result.solution.i_lo_ends)-1)
        i_hi_ends = (np.array(mzn_result.solution.i_hi_ends)-1)
        if len(i_hi_ends) > 0: 
            ix_lo_ends = wdf.index[i_lo_ends] 
            ix_hi_ends = wdf.index[i_hi_ends] 
    # print(f'i_spa_ends={i_spa_ends}')
    ###\ Minizinc


    plt.close('all')
    fig = plt.figure(figsize=(9,3))

    (ax1,ax2, ax3) = fig.subplots(3, 1, height_ratios=[3,1,1], sharex=True);

    plot_candles(wdf, ax=ax1) if candles else ax1.plot(wdf.close, lw=0.3, c='b')
    # ax1.plot(fu_mins,c='g')
    # ax1.plot(fu_maxs,c='r')
    # ax1.plot(sdf.ba_min, c='g', lw=0.2); ax1.plot(sdf.ba_max, c='r', lw=0.2)
    # sdf.ba_max.plot(ax=ax1, c='r')
    if len(ix_spa_sigs) > 0:
        spa_lquargs = {'lw':0.3,'color':'m'}
        ax1.vlines(ix_spa_begs, spa_mins, spa_maxs,**{**spa_lquargs})
        # ax1.vlines(ix_spa_sigs, spa_mins, spa_maxs,**{**spa_lquargs})
        # ax1.vlines(ix_spa_ends, spa_mins, spa_maxs,**{**spa_lquargs, 'linestyle':'--'})
        ax1.vlines(ix_spa_sigs, spa_mins, spa_maxs,**{**spa_lquargs, 'lw':1, 'alpha':0.7})
        ax1.hlines(spa_mins, ix_spa_begs, ix_spa_sigs, **spa_lquargs)
        ax1.hlines(spa_maxs, ix_spa_begs, ix_spa_sigs, **spa_lquargs)
        ax1.hlines(spa_mins, ix_spa_sigs, ix_lo_ends, **{**spa_lquargs, 'linestyle':'--'})
        ax1.hlines(spa_maxs, ix_spa_sigs, ix_hi_ends, **{**spa_lquargs, 'linestyle':'--'})

    plt.show()
    
out = widgets.interactive_output(printer, {
        **{k : v['wdg'] for k,v in all_params.items()}
    });
x = display(ui, out);


In [4]:
x = 1

In [2]:
y= x+1

In [3]:
y

3

P(B) = 0.0002
P(A|B) = P(B|A) * P(A) / P(B)
P(A|B) - 

In [84]:
0.85*0.002/(0.85 + 1 - 0.95)

0.0018888888888888885

P(A|B) * P(B) = P(B|A) * P(A)

P(fan belt|noises) = 0.9
P(muffler|noises) = 0.1
P(rattle|muffler) = 0.95
P(rattle|fan belt) = 0.08
P(muffler|rattle) = P(rattle|muffler) * P(muffle)

P(muffler|rattle) * P(rattle) = P(rattle|muffler) * P(muffer)
P(muffler|rattle) = P(rattle|muffler) * P(muffler) / P(rattle)

In [86]:
198 * 0.05

9.9